# Entrenamiento de word embeddings con EM

Voy a definir un modelo de variables latentes $\mathbf{z}_1,\ldots,\mathbf{z}_N$ con realizaciones en $\mathbb{R}^{|V|}$, que representan la palabra central, y variables observables $\mathbf{x}_1,\ldots,\mathbf{x}_N$ también con realizaciones en $\mathbb{R}^{|V|}$ que representan la palabra del contexto.

De esta forma, queda que la probabilidad de obtener en el contexto la palabra $o$ del vocabulario, dado que salió la palabra $c$ como palabra central, es:

$$
P(x_o = 1 | z_c = 1) = P(u_o|v_c) = Softmax(\mathcal{U}^Tv_c) = \frac{\exp(u_o^Tv_c)}{\sum_u \exp(u^Tv_c)}
$$

con $u_o$ y $v_c$ los word embeddings de las palabras $o$ y $c$ del vocabulario.

Definimos, además la probabilidad de obtener la palabra $c$ del vocabulario como palabra central:

$$
P(z_c = 1) = \pi_c
$$

de manera que la probabilidad $P(\mathbf{x})$ queda definida por la siguiente ecuación:

$$
P(x_o = 1) = \sum_{c=1}^{|V|} P(z_c = 1) P(x_o = 1 | z_c = 1) = \sum_{c=1}^{|V|} \pi_c P(u_o|v_c) \;\;\;\;\;\; o=1,\ldots,|V|
$$


Deducimos el algoritmo EM para este modelo. Dada una cantidad de muestras $\mathbf{x}_1,\ldots,\mathbf{x}_N$ La función log-likelihood queda definida por

$$
\begin{align*}
\mathcal{LL}(\mathbf{x}_1,\ldots,\mathbf{x}_N|u_1,\ldots,u_{|V|},v_1,\ldots,v_{|V|}) &=
\mathcal{LL}(\mathbf{x}_1,\ldots,\mathbf{x}_N|\theta) \\
&= \log \left( \prod_{n=1}^N P(\mathbf{x}_n | \theta) \right) \\
&=\sum_{n=1}^N \log\left(P(\mathbf{x}_n | \theta) \right) \\
&=\sum_{n=1}^N \log\left( \sum_{c=1}^{|V|} \pi_c P(u_{\mathbf{x_n}}|v_c) \right) 
\end{align*}
$$

Ahora, buscamos las ecuaciones del algoritmo EM para este modelo derivando con respecto a $v_j$ y a $u_j$ para todo $j=1,\ldots,|V|$:

$$
\begin{align*}
\frac{\partial \mathcal{LL}}{\partial v_j} &= \sum_{n=1}^N 
\frac{\sum_{c=1}^{|V|} \pi_c \frac{\partial}{\partial v_j} P(u_{\mathbf{x_n}}|v_c)}{\sum_{c=1}^{|V|} \pi_c P(u_{\mathbf{x_n}}|v_c)} \\[1em]
&= \sum_{n=1}^N \frac{\pi_j \frac{\partial}{\partial v_j} P(u_{\mathbf{x_n}}|v_j)}{\sum_{c=1}^{|V|} \pi_c P(u_{\mathbf{x_n}}|v_c)} \\
\end{align*}
$$

Obtenemos $\frac{\partial}{\partial v_j} P(u_o|v_j)$:

$$
\begin{align*}
\frac{\partial}{\partial v_j} P(u_o|v_j) &= \frac{\partial}{\partial v_j} \frac{\exp(u_o^Tv_j)}{\sum_{k=1}^{|V|} \exp(u_k^Tv_j)} \\[1em]
&= \frac{ \frac{\partial \exp(u_o^Tv_j)}{\partial v_j} \sum_{k=1}^{|V|} \exp(u_k^Tv_j) - 
\exp(u_o^Tv_j) \frac{\partial \sum_{k=1}^{|V|} \exp(u_k^Tv_j)}{\partial v_j}}{ \left(\sum_{k=1}^{|V|} \exp(u_k^Tv_j) \right)^2}\\[1em]
&= \frac{ u_o \exp(u_o^Tv_j) \sum_{k=1}^{|V|} \exp(u_k^Tv_j) - 
\exp(u_o^Tv_j) \sum_{k=1}^{|V|} u_k \exp(u_k^Tv_j) }{ \left(\sum_{k=1}^{|V|} \exp(u_k^Tv_j) \right)^2}\\[1em]
&= \frac{\exp(u_o^Tv_j)}{\sum_{k=1}^{|V|} u_k \exp(u_k^Tv_j)} 
\left( u_o - \sum_{k'=1}^{|V|} \frac{ u_{k'} \exp(u_{k'}^Tv_j)}{\sum_{k=1}^{|V|} \exp(u_k^Tv_j)}\right)\\[1em]
&= P(u_o|v_j) 
\sum_{k'=1}^{|V|} u_{k'} \delta_{ok'} - \frac{ u_{k'} \exp(u_{k'}^Tv_j)}{\sum_{k=1}^{|V|} \exp(u_k^Tv_j)}\\[1em]
&= P(u_o|v_j) \sum_{k'=1}^{|V|} \left(\delta_{ok'} - \frac{ \exp(u_{k'}^Tv_j)}{\sum_{k=1}^{|V|} \exp(u_k^Tv_j)} \right) u_{k'}\\[1em]
&= P(u_o|v_j) \mathcal{U} (\mathbf{y}_o - \hat{\mathbf{y}})
\end{align*}
$$

donde se definieron las coordenadas de $\mathbf{y}_o$ y de $\hat{\mathbf{y}}$, respectivamente, como

$$
\begin{align*}
y_i = \begin{cases} 1 & i=o \\ 0 & i\neq o \end{cases} & & 
\hat{y}_i = P(u_i|v_j) & \hspace{3em} & i=1,\ldots,|V|
\end{align*}
$$

